In [1]:
import pandas as pd
import numpy as np

In [16]:
def most_popular_tweets(df, tweet_date):
    ''' Find and return the most liked/ most retweeted positive and negative tweets overall and per party
    --> returns two df most_liked_tweets, most_retweeted_tweets '''

    #Create empty dataframe for most likes and most retweeted tweets
    most_liked_tweets = pd.DataFrame()
    most_retweeted_tweets = pd.DataFrame()

    # Create lists for iterations for party and sentiment
    parties = ['CDU', 'SPD', 'GRUENE', 'FDP', 'LINKE', 'AFD', 'OTHER']
    sentiments = [-2, 2]  #-2: negative, +2: positive

    #Columns to keep
    columns = [
        'username', 'text', 'tweet_date', 'retweet_count', 'reply_count', 'like_count',
        'sentiment', "party"
    ]
    #'username'

    #Only keep relevant columns of Twitter dataframe
    df = df[columns]

    #Change to datemtime
    df["tweet_date"] = pd.to_datetime(df["tweet_date"])

    #Change format of datetime to 2021-08-25
    df["tweet_date"] = df["tweet_date"].dt.strftime('%Y-%m-%d')

    # Filter df for day of tweet
    df = df[df["tweet_date"] == tweet_date]

    #Get most liked/ retweeted Tweets over all parties
    for sentiment in sentiments:

        #Filter for sentiment
        tweets_tmp = df[df["sentiment"] == sentiment].reset_index()

        # Find tweet with most likes and concatenate to final df
        most_liked_tweet = tweets_tmp.iloc[[tweets_tmp["like_count"].idxmax()]]
        most_liked_tweets = pd.concat([most_liked_tweets, most_liked_tweet],
                                      ignore_index=True)
        most_liked_tweets["party"] = "OVERALL"

        # Find tweet with most retweets and concatenate to final df
        most_retweeted_tweet = tweets_tmp.iloc[[
            tweets_tmp["retweet_count"].idxmax()
        ]]
        most_retweeted_tweets = pd.concat(
            [most_retweeted_tweets, most_retweeted_tweet], ignore_index=True)
        most_retweeted_tweets["party"] = "OVERALL"

    #Get most liked/ retweeted Tweets for each party
    for party in parties:
        
        #if party == "OTHER":
        #    import ipdb
        #    ipdb.set_trace()

        tweets_party_df = df[df["party"] == party]

        #Find tweets with most likes & most retweets per party and concatenate them to the final dfs
        for sentiment in sentiments:
            
            
            #Filter for sentiment
            tweets_tmp = pd.DataFrame(tweets_party_df[
                tweets_party_df["sentiment"] == sentiment].reset_index())
            

            
            # Find tweet with most likes and concatenate to final df
            most_liked_tweet = tweets_tmp.iloc[[
                tweets_tmp["like_count"].idxmax()
            ]]
            most_liked_tweets = pd.concat(
                [most_liked_tweets, most_liked_tweet], ignore_index=True)

            # Find tweet with most retweets and concatenate to final df
            most_retweeted_tweet = tweets_tmp.iloc[[
                tweets_tmp["retweet_count"].idxmax()
            ]]
            most_retweeted_tweets = pd.concat(
                [most_retweeted_tweets, most_retweeted_tweet],
                ignore_index=True)

    return most_liked_tweets, most_retweeted_tweets


In [8]:
tweet_date = "2021-08-31"

In [22]:
data1 = pd.read_csv("/Users/philippheitmann/Downloads/tweet_database_08_31.csv" , lineterminator="\n")

/Users/philippheitmann/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,2,6,7,8,9,10,16,17,18,19,20,21,22,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
from project_delphi.features import load_and_clean_csv

In [24]:
data = load_and_clean_csv(data1)

> /Users/philippheitmann/code/NicolasBuehringer/NicolasBuehringer/project_delphi/project_delphi/features.py(53)load_and_clean_csv()
     52     ipdb.set_trace()
---> 53     df['profile_creation_date'] = df['profile_creation_date'].str.slice(0,19)
     54     df["profile_creation_date"] = pd.to_datetime(df["profile_creation_date"])

ipdb> continue


In [29]:
data.shape

(268026, 15)

In [27]:
data.to_csv("../../raw_data/test_tweets.csv")

In [36]:
data.describe()

,sentiment
count,268026.000000
mean,-0.320353
std,1.559678
min,-2.000000
25%,-2.000000
50%,1.000000
75%,1.000000
max,2.000000


In [25]:
tweets_tmp = data[(data["sentiment"]==-2) & (data["party"]=="OTHER")].reset_index()
tweets_tmp.iloc[[tweets_tmp["like_count"].idxmax()]]

TypeError: reduction operation 'argmax' not allowed for this dtype

In [20]:
data

,Unnamed: 0,tweet_created_at,tweet_id,source,lang,text,author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,...,public_metrics.listed_count,withheld.copyright,withheld.country_codes_x,withheld.country_codes_y,party,withheld.scope_x,withheld.scope_y,withheld.country_codes,withheld.scope,sentiment
0,0,2021-08-31T23:59:24.000Z,1432855573284937730,Twitter Web App,de,Heftige Merkel-Attacke - ist Olaf Scholz eine ...,229106593,1,0,0,...,0,NaN,NaN,NaN,SPD,NaN,NaN,NaN,NaN,neutral
1,1,2021-08-31T23:58:48.000Z,1432855420482334725,Twitter for iPhone,de,@larsklingbeil Klingbeil und was ist mit der S...,3625766896,0,0,0,...,0,NaN,NaN,NaN,SPD,NaN,NaN,NaN,NaN,neutral
2,2,2021-08-31T23:58:42.000Z,1432855397963083780,Twitter Web App,de,"Hauptsächlich Esken das ref, spielt sich vor d...",1170234519951159296,0,0,1,...,3,NaN,NaN,NaN,SPD,NaN,NaN,NaN,NaN,neutral
3,3,2021-08-31T23:52:45.000Z,1432853899514421248,Twitter Web App,de,"Scholz ist blöder als lang..\nMister Teflon.,\...",1170234519951159296,1,1,1,...,3,NaN,NaN,NaN,SPD,NaN,NaN,NaN,NaN,neutral
4,4,2021-08-31T23:58:12.000Z,1432855271869661184,Twitter Web App,de,"""Menschen dürfen nicht GEGENEINANDER ausgespie...",1730034720,0,0,0,...,0,NaN,NaN,NaN,SPD,NaN,NaN,NaN,NaN,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276124,276124,2021-08-27T00:01:20.000Z,1431044121393836034,Twitter Web App,de,@abendblatt Herr Röhse hat jahrelang den ÖPNV...,18309034,0,0,0,...,3,NaN,NaN,NaN,CDU,NaN,NaN,NaN,NaN,negative
276125,276125,2021-08-27T00:03:59.000Z,1431044787705090048,Twitter for Android,de,Auch bei #Illner gab es also #KeineUnion #Wege...,49985382,1,0,6,...,2,NaN,NaN,NaN,CDU,NaN,NaN,NaN,NaN,neutral
276126,276126,2021-08-27T00:03:08.000Z,1431044573481086982,Twitter Web App,de,"@akk @cducsubt Man sollte sich langsam Fragen,...",1381013960917409800,0,0,0,...,0,NaN,NaN,NaN,CDU,NaN,NaN,NaN,NaN,neutral
276127,276127,2021-08-27T00:03:08.000Z,1431044571606179845,Blog2Social APP,de,"Foto: Armin Laschet, über dts Nachrichtenagent...",502657567,0,0,0,...,9,NaN,NaN,NaN,CDU,NaN,NaN,NaN,NaN,neutral


In [17]:
most_popular_tweets(data, tweet_date)

/var/folders/r8/ml9zphbs05z5wx9ykw9g7r_40000gn/T/ipykernel_5853/3445406669.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tweet_date"] = pd.to_datetime(df["tweet_date"])
/var/folders/r8/ml9zphbs05z5wx9ykw9g7r_40000gn/T/ipykernel_5853/3445406669.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tweet_date"] = df["tweet_date"].dt.strftime('%Y-%m-%d')


(     index         username  \
 0    94890        HHumorlos   
 1    98287           iBlali   
 2    94890        HHumorlos   
 3    98287           iBlali   
 4     8692     JanSchipmann   
 5     4957       RickGPunkt   
 6   212927   nikitheblogger   
 7   214526  gruenestuttgart   
 8   191949       whostamina   
 9   188064           sixtus   
 10  258728    NiemaMovassat   
 11  259700   SusanneFerschl   
 12   59383   nikitheblogger   
 13   62079  benedikt_kaiser   
 14  268690   HubertAiwanger   
 15  267780    Grund_gesetzt   
 
                                                  text  tweet_date  \
 0                 Wer CDU wählt, wählt Armin Laschet.  2021-08-31   
 1   Ich habe bisher nicht eine gute Sache über #La...  2021-08-31   
 2                 Wer CDU wählt, wählt Armin Laschet.  2021-08-31   
 3   Ich habe bisher nicht eine gute Sache über #La...  2021-08-31   
 4   Könnt ihr auch nicht mehr schlafen, weil euch ...  2021-08-31   
 5        Olaf Scholz be like: htt